In [1]:
import pandas as pd

In [2]:
use = pd.read_excel('mria_nl_sut.xlsx', sheet_name = 'USE', index_col = [0,1], header = [0,1])

In [3]:
sectors = ['C10T12','C13T15','C16','C17','C18','C19','C20','C21','C22','C23',
           'C24','C25','C26','C27','C28','C29','C30','C31_32','C33']

products = ['CPA_C10T12','CPA_C13T15','CPA_C16','CPA_C17','CPA_C18','CPA_C19','CPA_C20','CPA_C21','CPA_C22','CPA_C23',
           'CPA_C24','CPA_C25','CPA_C26','CPA_C27','CPA_C28','CPA_C29','CPA_C30','CPA_C31_32','CPA_C33']


In [4]:
use.head(2)

NL11                                                        \
                   A01    A02-03         B    C10T12        C13T15       C16   
NL11 CPA_A01  2.065564  0.002889  0.001013  6.101331  3.602690e-02  0.003112   
     CPA_A02  0.000011  0.002414  0.000031  0.000003  9.754773e-07  0.018906   

                                                    ...          NL42  \
                   C17           C18 C19       C20  ...           N77   
NL11 CPA_A01  0.009570  9.523472e-09   0  0.005936  ...  3.336237e-07   
     CPA_A02  0.002453  2.636549e-08   0  0.000548  ...  1.526736e-07   

                                                                        \
                       N78           N79    N80T82       O84       P85   
NL11 CPA_A01  9.864528e-07  2.479145e-04  0.062176  0.057273  0.000021   
     CPA_A02  2.082089e-07  5.649227e-08  0.000010  0.001918  0.000004   

                                                      
                   Q86    Q87_88       R-U    FinalD  
NL11 CPA_A01  0.029889  0.047085  0.010222  1.248630  
     CPA_A02  0.000006  0.000001  0.001854  0.001893  

[2 rows x 672 columns]

#### Step 1: Estimating the regional consumption of products from NL33 manufacturing sectors

In [5]:
regions = list(use.index.get_level_values(0).unique())
# region of nterest
roi = 'NL33'

In [6]:
ratio_df = pd.DataFrame(index = regions)

In [7]:
for i in range(len(regions)):
    
    r = regions[i]
    sliced_df = use.loc[(roi,products),(r,list(use.columns.get_level_values(1).unique()))]
    consumption_sum = sliced_df.sum().sum() / 365
    ratio_df.loc[r,'con_nl33_products'] = consumption_sum

In [8]:
ratio_df

,con_nl33_products
NL11,2.187784
NL12,1.286214
NL13,0.957846
NL21,4.120177
NL22,7.688473
NL23,0.633822
NL31,4.902723
NL32,10.698335
NL33,27.075492
NL34,2.056566


#### Step 2: Estimating the consumption of manufacturing sector of nl33 from different regions

In [9]:
for i in range(len(regions)):
    r = regions[i]
    sliced_df = use.loc[(r,list(use.index.get_level_values(1).unique())),(roi,sectors)]
    consumption_sum = sliced_df.sum().sum() /365
    ratio_df.loc[r,'con_nl33_sectors'] = consumption_sum

In [10]:
ratio_df

,con_nl33_products,con_nl33_sectors
NL11,2.187784,4.599147
NL12,1.286214,0.312065
NL13,0.957846,0.247934
NL21,4.120177,1.729083
NL22,7.688473,5.778732
NL23,0.633822,0.153176
NL31,4.902723,2.625638
NL32,10.698335,14.007168
NL33,27.075492,30.478159
NL34,2.056566,0.379466


#### Step 3: Estimating the value supplied to value_used_ratio

In [11]:
ratio_df['ratio'] = ratio_df['con_nl33_products'] / ratio_df['con_nl33_sectors']
ratio_df.head(12)

,con_nl33_products,con_nl33_sectors,ratio
NL11,2.187784,4.599147,0.475693
NL12,1.286214,0.312065,4.121617
NL13,0.957846,0.247934,3.863313
NL21,4.120177,1.729083,2.382869
NL22,7.688473,5.778732,1.330478
NL23,0.633822,0.153176,4.137873
NL31,4.902723,2.625638,1.867251
NL32,10.698335,14.007168,0.763776
NL33,27.075492,30.478159,0.888357
NL34,2.056566,0.379466,5.419631


#### Step 4: Estimating the reginal outputs

In [12]:
sup = pd.read_excel('mria_nl_sut.xlsx', sheet_name = 'SUP', index_col = [0,1], header = [0,1])

In [13]:
sliced_df = sup.loc[('NL33',list(sup.index.get_level_values(1).unique())),:]

In [14]:
for i in range(len(regions)):
    r = regions[i]
    sliced_df = sup.loc[(r,list(sup.index.get_level_values(1).unique())),:]
    consumption_sum = sliced_df.sum().sum() / 365
    ratio_df.loc[r,'output'] = consumption_sum

In [15]:
ratio_df.head(12)

,con_nl33_products,con_nl33_sectors,ratio,output
NL11,2.187784,4.599147,0.475693,128.724653
NL12,1.286214,0.312065,4.121617,102.806543
NL13,0.957846,0.247934,3.863313,78.347367
NL21,4.120177,1.729083,2.382869,224.615279
NL22,7.688473,5.778732,1.330478,396.650072
NL23,0.633822,0.153176,4.137873,70.725523
NL31,4.902723,2.625638,1.867251,319.292522
NL32,10.698335,14.007168,0.763776,814.746277
NL33,27.075492,30.478159,0.888357,846.717063
NL34,2.056566,0.379466,5.419631,77.829260


In [16]:
ratio_df = ratio_df.sort_values(by = 'output', ascending = False)

In [17]:
ratio_df

,con_nl33_products,con_nl33_sectors,ratio,output
NL33,27.075492,30.478159,0.888357,846.717063
NL32,10.698335,14.007168,0.763776,814.746277
NL41,11.389273,16.851892,0.675845,625.086219
NL22,7.688473,5.778732,1.330478,396.650072
NL31,4.902723,2.625638,1.867251,319.292522
NL42,5.577682,2.893372,1.927744,235.040045
NL21,4.120177,1.729083,2.382869,224.615279
NL11,2.187784,4.599147,0.475693,128.724653
NL12,1.286214,0.312065,4.121617,102.806543
NL13,0.957846,0.247934,3.863313,78.347367


In [18]:
ratio_df.to_excel('ratio_df.xlsx')